## BP Weekly Reporting
### Data Wrangling

Wrangle and repackage weekly data extracts from GA, Facebook and Google AdWords into a tidy dataset prepped for analysis.

In [1]:
# import packages
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from time import strptime
import datetime as dt
import os
import re

In [2]:
## read in FB export 9/24-9/30
Facebook = pd.read_csv('Bluprint_Weekly_Reporting_Template_Facebook.csv')
print(Facebook.info())
Facebook.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6663 entries, 0 to 6662
Data columns (total 14 columns):
Ad Set ID               6663 non-null float64
Ad Set Name             6663 non-null object
Ad ID                   6663 non-null float64
Ad Name                 6663 non-null object
Day                     6663 non-null object
Platform                6663 non-null object
Amount Spent (USD)      6663 non-null float64
Impressions             6663 non-null int64
Link Clicks             2188 non-null float64
3-Second Video Views    2453 non-null float64
Video Watches at 95%    1895 non-null float64
Website Conversions     1391 non-null float64
Reporting Starts        6663 non-null object
Reporting Ends          6663 non-null object
dtypes: float64(7), int64(1), object(6)
memory usage: 728.8+ KB
None


,Ad Set ID,Ad Set Name,Ad ID,Ad Name,Day,Platform,Amount Spent (USD),Impressions,Link Clicks,3-Second Video Views,Video Watches at 95%,Website Conversions,Reporting Starts,Reporting Ends
0,2.384300e+16,"Post: ""No needles, no problem! This rug is kni...",2.384300e+16,"Post: ""No needles, no problem! This rug is kni...",10/5/18,facebook,509.81,77325,761.0,40053.0,12367.0,59.0,10/5/18,10/5/18
1,2.384300e+16,"Post: ""No needles, no problem! This rug is kni...",2.384300e+16,"Post: ""No needles, no problem! This rug is kni...",10/4/18,facebook,490.19,71800,820.0,35606.0,12365.0,20.0,10/4/18,10/4/18
2,2.384300e+16,"Post: ""Last minute project? These faster-than-...",2.384300e+16,"Post: ""Last minute project? These faster-than-...",10/2/18,facebook,441.15,47117,1476.0,NaN,NaN,148.0,10/2/18,10/2/18
3,2.384300e+16,"Post: ""Last minute project? These faster-than-...",2.384300e+16,"Post: ""Last minute project? These faster-than-...",10/4/18,messenger,0.01,7,NaN,NaN,NaN,NaN,10/4/18,10/4/18
4,2.384300e+16,"Post: ""Last minute project? These faster-than-...",2.384300e+16,"Post: ""Last minute project? These faster-than-...",10/4/18,instagram,1.06,474,5.0,NaN,NaN,NaN,10/4/18,10/4/18


In [3]:
# Drop unneeded columns 
Facebook.drop(columns=['Ad Set ID', 'Ad ID', 'Reporting Starts', 'Reporting Ends'], axis=1, inplace=True)
Facebook.head()

,Ad Set Name,Ad Name,Day,Platform,Amount Spent (USD),Impressions,Link Clicks,3-Second Video Views,Video Watches at 95%,Website Conversions
0,"Post: ""No needles, no problem! This rug is kni...","Post: ""No needles, no problem! This rug is kni...",10/5/18,facebook,509.81,77325,761.0,40053.0,12367.0,59.0
1,"Post: ""No needles, no problem! This rug is kni...","Post: ""No needles, no problem! This rug is kni...",10/4/18,facebook,490.19,71800,820.0,35606.0,12365.0,20.0
2,"Post: ""Last minute project? These faster-than-...","Post: ""Last minute project? These faster-than-...",10/2/18,facebook,441.15,47117,1476.0,NaN,NaN,148.0
3,"Post: ""Last minute project? These faster-than-...","Post: ""Last minute project? These faster-than-...",10/4/18,messenger,0.01,7,NaN,NaN,NaN,NaN
4,"Post: ""Last minute project? These faster-than-...","Post: ""Last minute project? These faster-than-...",10/4/18,instagram,1.06,474,5.0,NaN,NaN,NaN


In [4]:
# Change column names 
Facebook.columns = ['Placement', 'Creative', 'Date', 'Platform', 'Cost', 'Impressions', 'Clicks', 'Views', 'Completed Views', 'Conversions']
Facebook.head(10)

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,"Post: ""No needles, no problem! This rug is kni...","Post: ""No needles, no problem! This rug is kni...",10/5/18,facebook,509.81,77325,761.0,40053.0,12367.0,59.0
1,"Post: ""No needles, no problem! This rug is kni...","Post: ""No needles, no problem! This rug is kni...",10/4/18,facebook,490.19,71800,820.0,35606.0,12365.0,20.0
2,"Post: ""Last minute project? These faster-than-...","Post: ""Last minute project? These faster-than-...",10/2/18,facebook,441.15,47117,1476.0,NaN,NaN,148.0
3,"Post: ""Last minute project? These faster-than-...","Post: ""Last minute project? These faster-than-...",10/4/18,messenger,0.01,7,NaN,NaN,NaN,NaN
4,"Post: ""Last minute project? These faster-than-...","Post: ""Last minute project? These faster-than-...",10/4/18,instagram,1.06,474,5.0,NaN,NaN,NaN
5,"Post: ""Last minute project? These faster-than-...","Post: ""Last minute project? These faster-than-...",10/4/18,facebook,317.30,36630,975.0,NaN,NaN,120.0
6,"Post: ""Last minute project? These faster-than-...","Post: ""Last minute project? These faster-than-...",10/3/18,messenger,0.05,6,NaN,NaN,NaN,NaN
7,"Post: ""Last minute project? These faster-than-...","Post: ""Last minute project? These faster-than-...",10/3/18,instagram,2.94,1301,11.0,NaN,NaN,NaN
8,"Post: ""Last minute project? These faster-than-...","Post: ""Last minute project? These faster-than-...",10/3/18,facebook,727.45,83448,2348.0,NaN,NaN,244.0
9,"Post: ""Last minute project? These faster-than-...","Post: ""Last minute project? These faster-than-...",10/2/18,messenger,0.22,41,NaN,NaN,NaN,NaN


In [9]:
# read in Google export
Google = pd.read_csv('Bluprint_Weekly_Reporting_Template_Google.csv')
Google.head()

/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,13,14,15,17,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Bluprint Weekly Reporting Template,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,"October 1, 2018 - October 7, 2018",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ad group,Headline,Headline 1,Headline 2,Expanded Text Ad Headline 3,Short headline,Long headline,Description,Expanded Text Ad Description 2,Description line 1,...,Path 2,Business name,Day,Currency,Cost,Impressions,Clicks,Views,Video played to 100%,Conversions
2,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/1/18,USD,$0.00,0,0,0,--,0
3,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/3/18,USD,$0.00,0,0,0,--,0
4,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/5/18,USD,$0.00,0,0,0,--,0


In [10]:
# bring header up from row index 1 and drop imported header and row index 0
Google.drop(Google.index[:1], inplace=True)
new_header = Google.iloc[0]
Google = Google[1:]
Google.columns = new_header
Google = Google.reset_index(drop=True)
Google.head()

1,Ad group,Headline,Headline 1,Headline 2,Expanded Text Ad Headline 3,Short headline,Long headline,Description,Expanded Text Ad Description 2,Description line 1,...,Path 2,Business name,Day,Currency,Cost,Impressions,Clicks,Views,Video played to 100%,Conversions
0,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/1/18,USD,$0.00,0,0,0,--,0
1,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/3/18,USD,$0.00,0,0,0,--,0
2,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/5/18,USD,$0.00,0,0,0,--,0
3,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/6/18,USD,$0.00,0,0,0,--,0
4,AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_AA__Res_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/4/18,USD,$0.00,0,0,0,--,0


In [11]:
# filter out unwanted columns
Google = Google.filter(['Ad group','Ad','Day','Cost','Impressions','Clicks','Views','Video played to 100%','Conversions'])
print(Google.info())
Google.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79702 entries, 0 to 79701
Data columns (total 9 columns):
Ad group                79702 non-null object
Ad                      79702 non-null object
Day                     79702 non-null object
Cost                    79702 non-null object
Impressions             79702 non-null object
Clicks                  79702 non-null object
Views                   79702 non-null object
Video played to 100%    79702 non-null object
Conversions             79702 non-null object
dtypes: object(9)
memory usage: 5.5+ MB
None


1,Ad group,Ad,Day,Cost,Impressions,Clicks,Views,Video played to 100%,Conversions
0,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/1/18,$0.00,0,0,0,--,0
1,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/3/18,$0.00,0,0,0,--,0
2,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/5/18,$0.00,0,0,0,--,0
3,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/6/18,$0.00,0,0,0,--,0
4,AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_AA__Res_...,--,10/4/18,$0.00,0,0,0,--,0


In [13]:
# rename columns
Google['Platform'] = 'Google'
Google.columns = ['Placement','Creative','Date','Cost','Impressions','Clicks','Views','Completed Views','Conversions','Platform']
Google.head()

,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions,Platform
0,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/1/18,$0.00,0,0,0,--,0,Google
1,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/3/18,$0.00,0,0,0,--,0,Google
2,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/5/18,$0.00,0,0,0,--,0,Google
3,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/6/18,$0.00,0,0,0,--,0,Google
4,AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_AA__Res_...,--,10/4/18,$0.00,0,0,0,--,0,Google


In [14]:
# create function to strip non-numeric characters from a column
def strp_nonnum(col):
    '''strip string column of non numeric characters, keeping only digits and decimals'''
    col = col.str.extract('([\d]+.?\d*)')
    return col

# strip certain columns of non-numeric characters
cols = ['Cost','Impressions','Clicks','Views','Completed Views','Conversions']
for col in cols:
    Google[col] = strp_nonnum(Google[col])
    Google[col] = Google[col].str.replace(',','').astype(float)

print(Google.info())
Google.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79702 entries, 0 to 79701
Data columns (total 10 columns):
Placement          79702 non-null object
Creative           79702 non-null object
Date               79702 non-null object
Cost               79702 non-null float64
Impressions        79702 non-null float64
Clicks             79702 non-null float64
Views              65534 non-null float64
Completed Views    279 non-null float64
Conversions        32766 non-null float64
Platform           79702 non-null object
dtypes: float64(6), object(4)
memory usage: 6.1+ MB
None


,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions,Platform
0,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/1/18,0.0,0.0,0.0,0.0,NaN,0.0,Google
1,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/3/18,0.0,0.0,0.0,0.0,NaN,0.0,Google
2,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/5/18,0.0,0.0,0.0,0.0,NaN,0.0,Google
3,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/6/18,0.0,0.0,0.0,0.0,NaN,0.0,Google
4,AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_AA__Res_...,--,10/4/18,0.0,0.0,0.0,0.0,NaN,0.0,Google


In [15]:
Google.Cost.sum()

121333.40999999997

In [16]:
Google = Google[['Placement','Creative','Date','Platform','Cost','Impressions','Clicks','Views','Completed Views','Conversions']]
Google.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/1/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
1,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/3/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
2,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/5/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
3,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/6/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
4,AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_AA__Res_...,--,10/4/18,Google,0.0,0.0,0.0,0.0,NaN,0.0


In [31]:
# read in Pinterest export
Pinterest = pd.read_csv('Bluprint_Weekly_Reporting_Template_Pinterest.csv')
Pinterest.head()

,Ad holdout experiment,Promoted Pin ID,Pin Promotion campaign id,Campaign name,Ad group name,Promoted Pin name,Pin,Promoted Pin description,Destination URL,Spend,...,Add to cart - return on ad spend,Add to cart - number of conversions,Add to cart - conversion value,Add to cart - click conversions,Add to cart - value from click conversions,Add to cart - engagement conversions,Add to cart - value from engagement conversions,Add to cart - view conversions,Add to cart - value from view conversions,Date
0,NaN,NaN,NaN,Cons_SP_Pin_SVOD_All_MULTISHOWCASE_Symphony_Video,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/1/18
1,NaN,NaN,NaN,Cons_SP_Pin_SVOD_All_MULTISHOWCASE_Symphony_Video,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/2/18
2,NaN,NaN,NaN,Cons_SP_Pin_SVOD_All_MULTISHOWCASE_Symphony_Video,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/3/18
3,NaN,NaN,NaN,Cons_SP_Pin_SVOD_All_MULTISHOWCASE_Symphony_Video,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/4/18
4,NaN,NaN,NaN,Cons_SP_Pin_SVOD_All_MULTISHOWCASE_Symphony_Video,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,NaN,NaN,NaN,82.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/5/18


In [32]:
# filter out unwanted columns, more efficient than dropping due to quantity of columns
Pinterest = Pinterest.filter(['Ad group name','Promoted Pin name','Spend','Total impressions','Total clicks','Total conversions','Date'])
Pinterest.head()

,Ad group name,Promoted Pin name,Spend,Total impressions,Total clicks,Total conversions,Date
0,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,0,0,0,0,10/1/18
1,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,0,0,0,0,10/2/18
2,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,0,0,0,0,10/3/18
3,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,0,0,0,0,10/4/18
4,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,82.39,9154,11,3,10/5/18


In [33]:
# rename columns
Pinterest.columns = ['Placement', 'Creative', 'Cost', 'Impressions', 'Clicks', 'Conversions', 'Date']
Pinterest.head()

,Placement,Creative,Cost,Impressions,Clicks,Conversions,Date
0,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,0,0,0,0,10/1/18
1,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,0,0,0,0,10/2/18
2,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,0,0,0,0,10/3/18
3,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,0,0,0,0,10/4/18
4,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,82.39,9154,11,3,10/5/18


In [34]:
# add columns to match FB df
Pinterest['Views'], Pinterest['Completed Views'] = [np.nan, np.nan]
Pinterest['Platform'] = 'Pinterest'
Pinterest.head()

,Placement,Creative,Cost,Impressions,Clicks,Conversions,Date,Views,Completed Views,Platform
0,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,0,0,0,0,10/1/18,NaN,NaN,Pinterest
1,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,0,0,0,0,10/2/18,NaN,NaN,Pinterest
2,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,0,0,0,0,10/3/18,NaN,NaN,Pinterest
3,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,0,0,0,0,10/4/18,NaN,NaN,Pinterest
4,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,82.39,9154,11,3,10/5/18,NaN,NaN,Pinterest


In [35]:
# reorder dataframe columns to make consistent with Facebook dataframe
Pinterest = Pinterest[['Placement','Creative','Date','Platform','Cost','Impressions','Clicks','Views','Completed Views','Conversions']]
Pinterest.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,10/1/18,Pinterest,0,0,0,NaN,NaN,0
1,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,10/2/18,Pinterest,0,0,0,NaN,NaN,0
2,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,10/3/18,Pinterest,0,0,0,NaN,NaN,0
3,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,10/4/18,Pinterest,0,0,0,NaN,NaN,0
4,Cons_SP_Pin_MULTISHOWCASE_All_All_Symphony_WCA...,All_Video_Symphony-MaxWidth-Original,10/5/18,Pinterest,82.39,9154,11,NaN,NaN,3


In [36]:
# read in Google Analytics export
GoogleAnalytics = pd.read_csv('Bluprint_Weekly_Reporting_Template_Google_Analytics_Raw.csv')
GoogleAnalytics.head(10)

,# ----------------------------------------,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,# A-01 Bluprint Web Prod - All,NaN,NaN,NaN,NaN,NaN,NaN
1,# LP Visitor Source_Weekly Reporting Pull,NaN,NaN,NaN,NaN,NaN,NaN
2,# 20181001-20181007,NaN,NaN,NaN,NaN,NaN,NaN
3,# ----------------------------------------,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Marketing Action Id,Date,Sessions,New Users,Bounces,Register (Goal 6 Completions),Subscribe (Goal 1 Completions)
6,114588,20181005,"57,169","47,253","48,465","2,633",3
7,114592,20181007,"49,457","38,352","42,215",262,0
8,114588,20181006,"46,212","39,634","32,747","4,706",15
9,114588,20181007,"45,943","39,641","30,692","5,606",11


In [37]:
# bring up row index 5 as header and drop row indices 0-4
GoogleAnalytics.drop(GoogleAnalytics.index[:5], inplace=True)
new_header = GoogleAnalytics.iloc[0]
GoogleAnalytics = GoogleAnalytics[1:]
GoogleAnalytics.columns = new_header
GoogleAnalytics = GoogleAnalytics.reset_index(drop=True)
GoogleAnalytics.head()

5,Marketing Action Id,Date,Sessions,New Users,Bounces,Register (Goal 6 Completions),Subscribe (Goal 1 Completions)
0,114588,20181005,"57,169","47,253","48,465","2,633",3
1,114592,20181007,"49,457","38,352","42,215",262,0
2,114588,20181006,"46,212","39,634","32,747","4,706",15
3,114588,20181007,"45,943","39,641","30,692","5,606",11
4,114592,20181006,"45,817","35,042","38,817",260,3


In [38]:
GoogleAnalytics.columns = ['MAID','Date','Sessions','New Users','Bounces','Registrations','Subscriptions']
print(GoogleAnalytics.head())
GoogleAnalytics.info()

     MAID      Date Sessions New Users Bounces Registrations Subscriptions
0  114588  20181005   57,169    47,253  48,465         2,633             3
1  114592  20181007   49,457    38,352  42,215           262             0
2  114588  20181006   46,212    39,634  32,747         4,706            15
3  114588  20181007   45,943    39,641  30,692         5,606            11
4  114592  20181006   45,817    35,042  38,817           260             3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3258 entries, 0 to 3257
Data columns (total 7 columns):
MAID             3258 non-null object
Date             3258 non-null object
Sessions         3258 non-null object
New Users        3258 non-null object
Bounces          3258 non-null object
Registrations    3258 non-null object
Subscriptions    3258 non-null object
dtypes: object(7)
memory usage: 178.2+ KB


In [39]:
# bring date column to datetime dtype
GoogleAnalytics['Date'] = pd.to_datetime(GoogleAnalytics['Date'], format = '%Y/%m/%d')

# strip non-numeric characters from certain columns
cols = ['Sessions', 'New Users', 'Bounces', 'Registrations', 'Subscriptions']
for col in cols:
    GoogleAnalytics[col] = strp_nonnum(GoogleAnalytics[col])
    GoogleAnalytics[col] = GoogleAnalytics[col].str.replace(',','')
    GoogleAnalytics[col] = pd.to_numeric(GoogleAnalytics[col], errors='coerce')
    
GoogleAnalytics.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions
0,114588,2018-10-05,57169,47253,48465,2633,3
1,114592,2018-10-07,49457,38352,42215,262,0
2,114588,2018-10-06,46212,39634,32747,4706,15
3,114588,2018-10-07,45943,39641,30692,5606,11
4,114592,2018-10-06,45817,35042,38817,260,3


In [40]:
# aggregate by MAID and Date, summing remaining columns
GoogleAnalytics = GoogleAnalytics.groupby(['MAID','Date']).sum().reset_index()
GoogleAnalytics.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions
0,0,2018-10-05,1,1,0,0,0
1,1.13938E+11,2018-10-01,4,4,3,0,0
2,1.13938E+11,2018-10-02,9,9,8,1,0
3,1.13938E+11,2018-10-03,1,1,1,0,0
4,1.13938E+11,2018-10-04,3,3,2,0,0


In [56]:
##MERGE DATA SETS
all_data = pd.concat([Google, Facebook, Pinterest], ignore_index=True)
all_data.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/1/18,Google,0,0.0,0.0,0.0,NaN,0.0
1,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/3/18,Google,0,0.0,0.0,0.0,NaN,0.0
2,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/5/18,Google,0,0.0,0.0,0.0,NaN,0.0
3,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/6/18,Google,0,0.0,0.0,0.0,NaN,0.0
4,AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_AA__Res_...,--,10/4/18,Google,0,0.0,0.0,0.0,NaN,0.0


In [57]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86393 entries, 0 to 86392
Data columns (total 10 columns):
Placement          86393 non-null object
Creative           86393 non-null object
Date               86393 non-null object
Platform           86393 non-null object
Cost               86393 non-null object
Impressions        86393 non-null float64
Clicks             81918 non-null float64
Views              67987 non-null float64
Completed Views    2174 non-null float64
Conversions        34185 non-null float64
dtypes: float64(5), object(5)
memory usage: 6.6+ MB


In [58]:
# create new columns and categorize
all_data['MAID'] = all_data['Placement'].str.extract('(_1\d\d\d\d\d)', expand=True)
all_data['MAID'] = all_data['MAID'].str.strip('_')
all_data['Funnel'] = all_data['Placement'].str.split('_').str[0]
#all_data['Tactic'] = all_data['Placement'].str.split('_').str[1]
all_data['Publisher'] = all_data['Placement'].str.split('_').str[2]
all_data['Hook'] = all_data['Placement'].str.split('_').str[3]
all_data['Mega'] = all_data['Placement'].str.split('_').str[4]
all_data['Micro'] = all_data['Placement'].str.split('_').str[5]
all_data['Campaign'] = all_data['Placement'].str.split('_').str[6]
all_data['Targ/aud/pin'] = all_data['Placement'].str.split('_').str[7]
all_data['Keyword'] = all_data['Placement'].str.split('_').str[8]
all_data.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,MAID,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword
0,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/1/18,Google,0,0.0,0.0,0.0,NaN,0.0,112074,AW,GDN,MULTISHOWCASE,Compose,TBD,-,AA,
1,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/3/18,Google,0,0.0,0.0,0.0,NaN,0.0,112074,AW,GDN,MULTISHOWCASE,Compose,TBD,-,AA,
2,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/5/18,Google,0,0.0,0.0,0.0,NaN,0.0,112074,AW,GDN,MULTISHOWCASE,Compose,TBD,-,AA,
3,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,10/6/18,Google,0,0.0,0.0,0.0,NaN,0.0,112074,AW,GDN,MULTISHOWCASE,Compose,TBD,-,AA,
4,AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_AA__Res_...,--,10/4/18,Google,0,0.0,0.0,0.0,NaN,0.0,112076,AW,GDN,MULTISHOWCASE,Stitch,TBD,-,AA,


In [59]:
all_data.groupby('Mega')['Cost'].nunique()

Mega
All                     1048
BP CA                     25
Bluprint LAL              87
Bluprint RM               88
Bluprint RM-Masthead      86
Bluprint SV               87
Compose                  705
Craftsy LAL              165
Craftsy SV               176
Dwell                    501
Engagement                 5
Lookalike Audience         1
Make                     548
Move                     603
Other                    177
Savor                    473
Site Visitors              5
Stitch                  1145
Together                 189
VV                         1
Name: Cost, dtype: int64

In [60]:
all_data['Funnel'].unique()

array(['AW', 'Cons', 'CONS', 'Angela2', 'Angela3', 'Embroidery1',
       'Embroidery3', 'Narwal3', 'Scott2', 'Scott3', 'Sewing2', 'Sewing4',
       'Sewing3', 'Narwal1', 'Angela5', 'Embroidery2', 'Embroidery4',
       'Angela1', 'Angela4', 'Embroidery5', 'Narwal4', 'Scott5', 'TVC3',
       'TVC4', 'TVC1', 'TVC2', 'Narwal2', 'Scott4', 'Narwal5', 'Scott1',
       'Sewing1', 'Angela3 open', 'Scott3 open', 'Scott5 open',
       'Sewing1 open', 'Sewing4 open', 'Embroidery2 open', 'TVC1 open',
       'TVC2 open', 'TVC3 open', 'CONV', 'Conv',
       'Post: "No needles, no problem! This rug is knit right on..."',
       'Post: "Last minute project? These faster-than-fast..."',
       'Post: "How to turn carrot cake into a delicious finger..."',
       'BST | Make a Connection | Parents | Test 3 | LPV |114525',
       'BST | Make a Connection | Not Parents | Test 3 | LPV |114525'],
      dtype=object)

In [61]:
all_data['Publisher'].unique()

array(['GDN', 'G', 'YT', nan, 'FB', 'Pin'], dtype=object)

In [62]:
# rename categories for Funnel and Publisher columns
all_data['Funnel'] = all_data['Funnel'].replace({'AW' : 'Awareness', 'Cons' : 'Consideration', 'CONS' : 'Consideration', 'Conv' : 'Conversion', 'CONV' : 'Conversion'})
all_data['Publisher'] = all_data['Publisher'].replace({'G' : 'Google Search', 'GDN' : 'Google Display Network', 'YT' : 'YouTube', 'FB' : 'Facebook', 'Pin' : 'Pinterest'})
all_data['Mega'] = all_data['Mega'].replace({'Site Visitors':'All/Other','Engagement':'All/Other','Other':'All/Other','Lookalike Audience':'All/Other','VV':'All/Other','Craftsy LAL':'All/Other','Craftsy SV':'All/Other','All':'All/Other','BP CA':'All/Other'})
print(all_data['Funnel'].unique())
print(all_data['Publisher'].unique())
all_data['Mega'].unique()

['Awareness' 'Consideration' 'Angela2' 'Angela3' 'Embroidery1'
 'Embroidery3' 'Narwal3' 'Scott2' 'Scott3' 'Sewing2' 'Sewing4' 'Sewing3'
 'Narwal1' 'Angela5' 'Embroidery2' 'Embroidery4' 'Angela1' 'Angela4'
 'Embroidery5' 'Narwal4' 'Scott5' 'TVC3' 'TVC4' 'TVC1' 'TVC2' 'Narwal2'
 'Scott4' 'Narwal5' 'Scott1' 'Sewing1' 'Angela3 open' 'Scott3 open'
 'Scott5 open' 'Sewing1 open' 'Sewing4 open' 'Embroidery2 open'
 'TVC1 open' 'TVC2 open' 'TVC3 open' 'Conversion'
 'Post: "No needles, no problem! This rug is knit right on..."'
 'Post: "Last minute project? These faster-than-fast..."'
 'Post: "How to turn carrot cake into a delicious finger..."'
 'BST | Make a Connection | Parents | Test 3 | LPV |114525'
 'BST | Make a Connection | Not Parents | Test 3 | LPV |114525']
['Google Display Network' 'Google Search' 'YouTube' nan 'Facebook'
 'Pinterest']


array(['Compose', 'Stitch', 'All/Other', 'Make', 'Savor', 'Dwell', 'Move',
       'Together', nan, 'Bluprint LAL', 'Bluprint SV', 'Bluprint RM',
       'Bluprint RM-Masthead'], dtype=object)

In [63]:
all_data['Mega'] = all_data['Mega'].replace({'Bluprint LAL':'All/Other','Bluprint SV':'All/Other','Bluprint RM':'All/Other','Bluprint RM-Masthead':'All/Other'})
all_data['Mega'].unique()

array(['Compose', 'Stitch', 'All/Other', 'Make', 'Savor', 'Dwell', 'Move',
       'Together', nan], dtype=object)

In [64]:
all_data.loc[(all_data['Funnel'] != 'Awareness') & (all_data['Funnel'] != 'Consideration') & (all_data['Funnel'] != 'Conversion')]

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,MAID,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword
26759,Angela2,Angela2,10/1/18,Google,0,0.0,0.0,0.0,NaN,0.0,NaN,Angela2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26760,Angela2,Angela2,10/2/18,Google,0,0.0,0.0,0.0,NaN,0.0,NaN,Angela2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26761,Angela2,Angela2,10/4/18,Google,0,0.0,0.0,0.0,NaN,0.0,NaN,Angela2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26762,Angela2,Angela2,10/6/18,Google,0,0.0,0.0,0.0,NaN,0.0,NaN,Angela2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26763,Angela2,Angela2,10/7/18,Google,0,0.0,0.0,0.0,NaN,0.0,NaN,Angela2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26764,Angela3,Angela3,10/1/18,Google,0,0.0,0.0,0.0,NaN,0.0,NaN,Angela3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26765,Angela3,Angela3,10/2/18,Google,0,0.0,0.0,0.0,NaN,0.0,NaN,Angela3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26766,Angela3,Angela3,10/3/18,Google,0,0.0,0.0,0.0,NaN,0.0,NaN,Angela3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26767,Angela3,Angela3,10/7/18,Google,0,0.0,0.0,0.0,NaN,0.0,NaN,Angela3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26768,Embroidery1,Embroidery1,10/1/18,Google,0,0.0,0.0,0.0,NaN,0.0,NaN,Embroidery1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
all_data = all_data.loc[(all_data['Funnel'] == 'Awareness') | (all_data['Funnel'] == 'Consideration') | (all_data['Funnel'] == 'Conversion')]
all_data.loc[(all_data1['Funnel'] != 'Awareness') & (all_data['Funnel'] != 'Consideration') & (all_data['Funnel'] != 'Conversion')]

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,MAID,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword


In [67]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85890 entries, 0 to 86392
Data columns (total 19 columns):
Placement          85890 non-null object
Creative           85890 non-null object
Date               85890 non-null object
Platform           85890 non-null object
Cost               85890 non-null object
Impressions        85890 non-null float64
Clicks             81431 non-null float64
Views              67504 non-null float64
Completed Views    2146 non-null float64
Conversions        33937 non-null float64
MAID               85043 non-null object
Funnel             85890 non-null object
Publisher          85890 non-null object
Hook               85890 non-null object
Mega               85890 non-null object
Micro              85890 non-null object
Campaign           85890 non-null object
Targ/aud/pin       85848 non-null object
Keyword            85848 non-null object
dtypes: float64(5), object(14)
memory usage: 13.1+ MB


In [68]:
Facebook['Platform'].unique()

array(['facebook', 'messenger', 'instagram', 'audience_network'],
      dtype=object)

In [69]:
all_data.loc[all_data['Platform'] == 'instagram', 'Publisher'] = 'Instagram'
all_data['Publisher'].unique()

array(['Google Display Network', 'Google Search', 'YouTube', 'Facebook',
       'Instagram', 'Pinterest'], dtype=object)

In [70]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85890 entries, 0 to 86392
Data columns (total 19 columns):
Placement          85890 non-null object
Creative           85890 non-null object
Date               85890 non-null object
Platform           85890 non-null object
Cost               85890 non-null object
Impressions        85890 non-null float64
Clicks             81431 non-null float64
Views              67504 non-null float64
Completed Views    2146 non-null float64
Conversions        33937 non-null float64
MAID               85043 non-null object
Funnel             85890 non-null object
Publisher          85890 non-null object
Hook               85890 non-null object
Mega               85890 non-null object
Micro              85890 non-null object
Campaign           85890 non-null object
Targ/aud/pin       85848 non-null object
Keyword            85848 non-null object
dtypes: float64(5), object(14)
memory usage: 13.1+ MB


In [77]:
all_data.loc[all_data['Placement'].str.split('_').str[1] == 'YT', 'Publisher'] = 'YouTube'

array(['AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res_20180717_-Art&Theater_112074',
       'AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_AA__Res_20180717_-AvidNewsReader_112076',
       'AW_DIS_GDN_MULTISHOWCASE_Dwell_TBD_-_AA__Res_20180717_-HomeEnthus_112073',
       'AW_DIS_GDN_MULTISHOWCASE_Make_TBD_-_AA__Res_20180717_-BeautyMavens_112075',
       'AW_DIS_GDN_MULTISHOWCASE_Savor_TBD_-_AA__Res_20180717_-CookingEnth_112078',
       'AW_DIS_GDN_MULTISHOWCASE_Dwell_TBD_-_IM__Res_20180717_-OutdoorItems_112073',
       'AW_DIS_GDN_MULTISHOWCASE_Dwell_TBD_-_IM__Res_20180717_-PartySupplies_112073',
       'AW_DIS_GDN_MULTISHOWCASE_Dwell_TBD_-_IM__Res_20180717_-HomeDecor_112073',
       'AW_DIS_GDN_MULTISHOWCASE_Dwell_TBD_-_IM__Res_20180717_-HomeFurnishing_112073',
       'AW_DIS_GDN_MULTISHOWCASE_Dwell_TBD_-_IM__Res_20180717_-HomeImprovement_112073',
       'AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_IM__Res_20180717_-Bedding_112076',
       'AW_DIS_GDN_MULTISHOWCASE_Stitch_TBD_-_AA__Res_20180717_-Ba

In [78]:
# calculate daily impressions by MAID
imp_by_MAID = all_data.groupby(by=['MAID', 'Date'])['Impressions'].sum().reset_index()
imp_by_MAID.head()

,MAID,Date,Impressions
0,111868,10/1/18,107.0
1,111868,10/2/18,98.0
2,111868,10/3/18,152.0
3,111868,10/4/18,134.0
4,111868,10/5/18,325.0


In [79]:
imp_by_MAID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1150 entries, 0 to 1149
Data columns (total 3 columns):
MAID           1150 non-null object
Date           1150 non-null object
Impressions    1150 non-null float64
dtypes: float64(1), object(2)
memory usage: 27.0+ KB


In [80]:
GoogleAnalytics.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions
0,0,2018-10-05,1,1,0,0,0
1,1.13938E+11,2018-10-01,4,4,3,0,0
2,1.13938E+11,2018-10-02,9,9,8,1,0
3,1.13938E+11,2018-10-03,1,1,1,0,0
4,1.13938E+11,2018-10-04,3,3,2,0,0


In [81]:
GoogleAnalytics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3257 entries, 0 to 3256
Data columns (total 7 columns):
MAID             3257 non-null object
Date             3257 non-null datetime64[ns]
Sessions         3257 non-null int64
New Users        3257 non-null int64
Bounces          3257 non-null int64
Registrations    3257 non-null int64
Subscriptions    3257 non-null int64
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 178.2+ KB


In [82]:
# merge imp_by_MAID and GoogleAnalytics
imp_by_MAID['Date'] = pd.to_datetime(imp_by_MAID['Date'])
GoogleAnalytics = pd.merge(GoogleAnalytics, imp_by_MAID, how='inner', on=['MAID', 'Date'])
print(GoogleAnalytics.head())
GoogleAnalytics.info()

     MAID       Date  Sessions  New Users  Bounces  Registrations  \
0  111868 2018-10-01        54         36       21              2   
1  111868 2018-10-02        26         15       13              2   
2  111868 2018-10-03        39         27       15              1   
3  111868 2018-10-04        42         31       15              2   
4  111868 2018-10-05       107         87       29              9   

   Subscriptions  Impressions  
0              1        107.0  
1              0         98.0  
2              1        152.0  
3              0        134.0  
4              0        325.0  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 888
Data columns (total 8 columns):
MAID             889 non-null object
Date             889 non-null datetime64[ns]
Sessions         889 non-null int64
New Users        889 non-null int64
Bounces          889 non-null int64
Registrations    889 non-null int64
Subscriptions    889 non-null int64
Impressions      889 non-nu

In [83]:
##CALCULATE COLUMNS TO BE JOINED INTO OVERALL DATASET
GoogleAnalytics['Session Rate'] = GoogleAnalytics['Sessions'].div(GoogleAnalytics['Impressions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['New Session Rate'] = GoogleAnalytics['New Users'].div(GoogleAnalytics['Sessions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['Bounce Rate'] = GoogleAnalytics['Bounces'].div(GoogleAnalytics['Sessions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['Reg Rate'] = GoogleAnalytics['Registrations'].div(GoogleAnalytics['Impressions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['Sub Rate'] = GoogleAnalytics['Subscriptions'].div(GoogleAnalytics['Impressions'], fill_value=0).replace({np.inf: 0})
GoogleAnalyticsRates = GoogleAnalytics
GoogleAnalyticsRates.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions,Impressions,Session Rate,New Session Rate,Bounce Rate,Reg Rate,Sub Rate
0,111868,2018-10-01,54,36,21,2,1,107.0,0.504673,0.666667,0.388889,0.018692,0.009346
1,111868,2018-10-02,26,15,13,2,0,98.0,0.265306,0.576923,0.500000,0.020408,0.000000
2,111868,2018-10-03,39,27,15,1,1,152.0,0.256579,0.692308,0.384615,0.006579,0.006579
3,111868,2018-10-04,42,31,15,2,0,134.0,0.313433,0.738095,0.357143,0.014925,0.000000
4,111868,2018-10-05,107,87,29,9,0,325.0,0.329231,0.813084,0.271028,0.027692,0.000000


In [84]:
GoogleAnalyticsRates.drop(columns=['Sessions', 'New Users', 'Bounces', 'Registrations', 'Subscriptions', 'Impressions'], axis=1, inplace=True)
GoogleAnalyticsRates.head()

,MAID,Date,Session Rate,New Session Rate,Bounce Rate,Reg Rate,Sub Rate
0,111868,2018-10-01,0.504673,0.666667,0.388889,0.018692,0.009346
1,111868,2018-10-02,0.265306,0.576923,0.500000,0.020408,0.000000
2,111868,2018-10-03,0.256579,0.692308,0.384615,0.006579,0.006579
3,111868,2018-10-04,0.313433,0.738095,0.357143,0.014925,0.000000
4,111868,2018-10-05,0.329231,0.813084,0.271028,0.027692,0.000000


In [85]:
##JOIN NEW GOOGLE ANALYTICS TO ALL_DATA_MERGE
all_data['Date'] = pd.to_datetime(all_data['Date'])
final_data_merge = pd.merge(all_data, GoogleAnalyticsRates, how='inner', on=['MAID', 'Date'])
print(final_data_merge.info())
final_data_merge.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74312 entries, 0 to 74311
Data columns (total 24 columns):
Placement           74312 non-null object
Creative            74312 non-null object
Date                74312 non-null datetime64[ns]
Platform            74312 non-null object
Cost                74312 non-null object
Impressions         74312 non-null float64
Clicks              69858 non-null float64
Views               57971 non-null float64
Completed Views     2130 non-null float64
Conversions         30754 non-null float64
MAID                74312 non-null object
Funnel              74312 non-null object
Publisher           74312 non-null object
Hook                74312 non-null object
Mega                74312 non-null object
Micro               74312 non-null object
Campaign            74312 non-null object
Targ/aud/pin        74312 non-null object
Keyword             74312 non-null object
Session Rate        74312 non-null float64
New Session Rate    74312 non-null flo

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,...,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Session Rate,New Session Rate,Bounce Rate,Reg Rate,Sub Rate
0,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,2018-10-01,Google,0,0.0,0.0,0.0,NaN,0.0,...,Compose,TBD,-,AA,,0.013161,0.679901,0.822837,0.000094,0.000007
1,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_IM__Res...,--,2018-10-01,Google,0,0.0,0.0,0.0,NaN,0.0,...,Compose,TBD,-,IM,,0.013161,0.679901,0.822837,0.000094,0.000007
2,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_IM__Res...,--,2018-10-01,Google,0,0.0,0.0,0.0,NaN,0.0,...,Compose,TBD,-,IM,,0.013161,0.679901,0.822837,0.000094,0.000007
3,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_IM__Res...,--,2018-10-01,Google,0,0.0,0.0,0.0,NaN,0.0,...,Compose,TBD,-,IM,,0.013161,0.679901,0.822837,0.000094,0.000007
4,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_IM__Res...,--,2018-10-01,Google,0,0.0,0.0,0.0,NaN,0.0,...,Compose,TBD,-,IM,,0.013161,0.679901,0.822837,0.000094,0.000007


In [86]:
# create new columns by multiplying others
final_data_merge['Sessions'] = final_data_merge['Impressions'].mul(final_data_merge['Session Rate'])
final_data_merge['New Sessions'] = final_data_merge['Sessions'].mul(final_data_merge['New Session Rate'])
final_data_merge['Bounces'] = final_data_merge['Sessions'].mul(final_data_merge['Bounce Rate'])
final_data_merge['Registrations'] = final_data_merge['Impressions'].mul(final_data_merge['Reg Rate'])
final_data_merge['Subscriptions'] = final_data_merge['Impressions'].mul(final_data_merge['Sub Rate'])
final_data_merge.drop(columns=['Session Rate', 'New Session Rate', 'Bounce Rate', 'Reg Rate', 'Sub Rate'], axis=1, inplace=True)
final_data_merge.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,...,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Sessions,New Sessions,Bounces,Registrations,Subscriptions
0,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_AA__Res...,--,2018-10-01,Google,0,0.0,0.0,0.0,NaN,0.0,...,Compose,TBD,-,AA,,0.0,0.0,0.0,0.0,0.0
1,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_IM__Res...,--,2018-10-01,Google,0,0.0,0.0,0.0,NaN,0.0,...,Compose,TBD,-,IM,,0.0,0.0,0.0,0.0,0.0
2,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_IM__Res...,--,2018-10-01,Google,0,0.0,0.0,0.0,NaN,0.0,...,Compose,TBD,-,IM,,0.0,0.0,0.0,0.0,0.0
3,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_IM__Res...,--,2018-10-01,Google,0,0.0,0.0,0.0,NaN,0.0,...,Compose,TBD,-,IM,,0.0,0.0,0.0,0.0,0.0
4,AW_DIS_GDN_MULTISHOWCASE_Compose_TBD_-_IM__Res...,--,2018-10-01,Google,0,0.0,0.0,0.0,NaN,0.0,...,Compose,TBD,-,IM,,0.0,0.0,0.0,0.0,0.0


In [87]:
final_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74312 entries, 0 to 74311
Data columns (total 24 columns):
Placement          74312 non-null object
Creative           74312 non-null object
Date               74312 non-null datetime64[ns]
Platform           74312 non-null object
Cost               74312 non-null object
Impressions        74312 non-null float64
Clicks             69858 non-null float64
Views              57971 non-null float64
Completed Views    2130 non-null float64
Conversions        30754 non-null float64
MAID               74312 non-null object
Funnel             74312 non-null object
Publisher          74312 non-null object
Hook               74312 non-null object
Mega               74312 non-null object
Micro              74312 non-null object
Campaign           74312 non-null object
Targ/aud/pin       74312 non-null object
Keyword            74312 non-null object
Sessions           74312 non-null float64
New Sessions       74312 non-null float64
Bounces         

In [88]:
##ADD REMAINING COLUMNS
final_data_merge['Start Date'] = final_data_merge.groupby(['Placement'])['Date'].transform(min)
final_data_merge['Video v Static'] = np.where(final_data_merge['Views']>=1, 'Video', 'Static')
final_data_merge['Impressions with Video View'] = np.where(final_data_merge['Views']>=1, final_data_merge['Impressions'], 0)
final_data_merge['Reporting Month'] = final_data_merge['Date'].dt.strftime('%B')
final_data_merge['Week Start'] = final_data_merge['Date'].dt.to_period('W').apply(lambda r: r.start_time)
final_data_merge['Reporting Period'] = final_data_merge['Date'].dt.to_period('W').apply(lambda r: r.start_time)

In [89]:
final_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74312 entries, 0 to 74311
Data columns (total 30 columns):
Placement                      74312 non-null object
Creative                       74312 non-null object
Date                           74312 non-null datetime64[ns]
Platform                       74312 non-null object
Cost                           74312 non-null object
Impressions                    74312 non-null float64
Clicks                         69858 non-null float64
Views                          57971 non-null float64
Completed Views                2130 non-null float64
Conversions                    30754 non-null float64
MAID                           74312 non-null object
Funnel                         74312 non-null object
Publisher                      74312 non-null object
Hook                           74312 non-null object
Mega                           74312 non-null object
Micro                          74312 non-null object
Campaign                       

In [90]:
#REORGANIZE REMAINING COLUMNS
final_data_merge = final_data_merge[['Funnel', 'Publisher', 'Hook', 'Mega', 'Micro', 
                                     'Campaign', 'Targ/aud/pin', 'Keyword', 'Start Date', 'MAID', 
                                     'Video v Static', 'Impressions with Video View', 'Week Start', 
                                     'Reporting Month', 'Reporting Period', 'Placement', 'Creative', 'Date', 
                                     'Cost', 'Impressions', 'Clicks', 'Views', 'Completed Views', 'Conversions', 
                                     'Sessions', 'New Sessions', 'Bounces', 'Registrations', 'Subscriptions']]
final_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74312 entries, 0 to 74311
Data columns (total 29 columns):
Funnel                         74312 non-null object
Publisher                      74312 non-null object
Hook                           74312 non-null object
Mega                           74312 non-null object
Micro                          74312 non-null object
Campaign                       74312 non-null object
Targ/aud/pin                   74312 non-null object
Keyword                        74312 non-null object
Start Date                     74312 non-null datetime64[ns]
MAID                           74312 non-null object
Video v Static                 74312 non-null object
Impressions with Video View    74312 non-null float64
Week Start                     74312 non-null datetime64[ns]
Reporting Month                74312 non-null object
Reporting Period               74312 non-null datetime64[ns]
Placement                      74312 non-null object
Creative          

In [91]:
final_data_merge['Date'].unique()

array(['2018-10-01T00:00:00.000000000', '2018-10-03T00:00:00.000000000',
       '2018-10-05T00:00:00.000000000', '2018-10-06T00:00:00.000000000',
       '2018-10-04T00:00:00.000000000', '2018-10-07T00:00:00.000000000',
       '2018-10-02T00:00:00.000000000'], dtype='datetime64[ns]')

In [92]:
final_data_merge['Targ/aud/pin'].unique()

array(['AA', 'IM', 'EG', 'JessicaLong', 'Celebrity', 'Celebrrity',
       'Generic', 'E', 'BrookeLark', 'JoshuaVergara', 'Doodle-Wars',
       'FabioViviani', 'JoshuaJohnRussell', 'BMM', 'SV', 'LAL', 'vv',
       'SV LAL', 'VV', 'im', 'aa', 'CA', 'BPRM', 'RM', 'Accessories',
       'WCA', 'Interest', 'BluprintLAL', 'RT', 'Demo'], dtype=object)

In [94]:
final_data_merge.to_csv('weekly_data_merge.csv')

### To Do


✅ drop 'Tactic' column 

✅ rename 'Video Completion' to 'Completed Views'

- audit columns for:
    - data type
    - quality
    - etc.
    
✅ Refine 'Mega' column to only include unique values, not duplicates with slight variations OR assign "All/Other" properly

In [95]:
final_data_merge.head()

,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Start Date,MAID,...,Impressions,Clicks,Views,Completed Views,Conversions,Sessions,New Sessions,Bounces,Registrations,Subscriptions
0,Awareness,Google Display Network,MULTISHOWCASE,Compose,TBD,-,AA,,2018-10-01,112074,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,Awareness,Google Display Network,MULTISHOWCASE,Compose,TBD,-,IM,,2018-10-01,112074,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,Awareness,Google Display Network,MULTISHOWCASE,Compose,TBD,-,IM,,2018-10-01,112074,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,Awareness,Google Display Network,MULTISHOWCASE,Compose,TBD,-,IM,,2018-10-01,112074,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,Awareness,Google Display Network,MULTISHOWCASE,Compose,TBD,-,IM,,2018-10-01,112074,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
